# Structural Lung Cancer Growth Model in Absence of Treatment

The tumour growth inhibitory effects of Erlotinib and Gefitinib were modelled with a population PKPD model in [1]. A population PKPD model is a hierarchical model which consists of a structural model, a population model, and an error model. Each sub-model captures a different aspect of the tumour growth inhibition biology, and is parametrised by a set of parameters. 

In this notebook we introduce the structural model for the tumour growth in absence of treatment proposed in [1]. 

## Structural growth model in absence of treatment

In a population PKPD model the structural model is a mechanistic or empirical description of the drug's pharmacokinetics (PK) and pharmacodynamics (PD). The PK part of the model describes how the drug is distributed in the body upon administration, which is often referred to as "what the body does to the drug". The PD part of the model captures the drug's effects on the progression of the disease, or "what the drug does to the body". 'Progression of the disease' may be quantified by any disease-related observable, such as biomarkers or in this case the tumour volume. A PD model does therefore not only need to capture the disease progression of an individual *under the influence* of a compound, but also needs to be able to describe its progresion in the *absence* of any treatment.

In [1] a structural model is proposed for the tumour growth in absence of the compound, which assumes that the tumour grows exponentially for small tumour volumes, and linearly for large tumour volumes

\begin{equation*}
    \frac{\text{d}V^s_T}{\text{d}t} = \frac{2\lambda _0\lambda _1 V^s_T}{2\lambda _0 V^s_T + \lambda _1}.
\end{equation*}

Here, $V^s_T$ is the predicted tumour volume by the structural model, $\lambda _0$ is the exponential growth rate, and $\lambda _1$ is the linear growth rate. The tumour growth of an individual in absence of the drug is thus parameterised by three parameters

\begin{equation*}
    \psi = (V_0, \lambda _0, \lambda _1),
\end{equation*}

where $V_0$ is the initial tumour volume, $V^s_T(t=0, \psi) = V_0$.

This model makes a number of assumptions and approximations to describe the tumour growth. The most obvious assumption of the model is a transition of the tumour growth from an exponential growth to a linear growth at a characteristic tumour volume

\begin{equation*}
    V_{\text{crit}} = \frac{\lambda _1}{2\lambda _0}.
\end{equation*}

For $V^s_T \ll V_{\text{crit}}$ the tumour is modelled by an exponential growth, while for $V^s_T \gg V_{\text{crit}}$ the tumour growth is linear. This tumour growth model was first introduced in [2], and builds on the intuition that in the early stages of the tumour an abundance of oxygen and nutrients leads to a constant doubling time of cancerous cells, and therefore to an exponential growth. However, in later stages of the tumour growth oxygen and other sources of nutrition are being depleted inside the tumour, and only the 'shell' tumour cells are able to proliferate at the inital rate. While there are ways for the tumour mass to expand in the inside too, by changing the mode of metabolism to glycolysis or rewiring of the blood vessels to improve the oxygen supply [3, 4, 5], it is clear that the total growth rate of the tumour should be expected to slow down. Due to the complexity of the process it is not obvious that the growth should be expected to change qualitatively from exponential to linear. However, in [2] it was argued that a linear growth phase was observed for later stages of tumour evolution in experiments. By investigating the estimate for $V_{\text{crit}}$ we will be able to assess this modelling choice directly.

## Visualisation of the structural model

In [33]:
#
# Create a visualisation of the structural model with parameter sliders.
#

from ipywidgets import widgets
import pandas as pd
import plotly.graph_objects as go

import pkpd


# Import data
path = os.getcwd()
data = pd.read_csv(path + '/data/lxf_control_growth.csv')

# Define model
path = pkpd.ModelLibrary().get_path('Tumour growth without treatment - Eigenmann et. al.')
model = pkpd.PharmacodynamicModel(path, is_log_transformed=False)

# Create figure widget
fig = go.FigureWidget()

# TODO: Generalise! Create mask for mouse
id_m = 40
mask = data['#ID'] == id_m

# Get observed data for indiviudal
observed_times = data['TIME in day'][mask].to_numpy()
observed_data = data['TUMOUR VOLUME in cm^3'][mask]

# Simulate data
params = [0.1, 0.1, 0.1]
start_experiment = data['TIME in day'].min()
end_experiment = data['TIME in day'].max()
simulated_times = np.linspace(
    start=start_experiment, stop=end_experiment)
simulated_data = model.simulate(params, simulated_times)

# Add time series plot
fig.add_trace(go.Scatter(
    x=simulated_times,
    y=simulated_data,
    legendgroup="ID: %d" % id_m,
    name="ID: %d" % id_m,
    showlegend=False,
    hovertemplate=(
        "<b>Simulation </b><br>" +
        "ID: %d<br>" % id_m +
        "Time: %{x:.0f} day<br>" +
        "Tumour volume: %{y:.02f} cm^3<br>" +
        "Cancer type: LXF A677<br>" +
        "<br>" +
        "<b>Parameter estimates </b><br>" +
        "Initial tumour volume: %.02f cm^3<br>" % params[0] +
        "Expon. growth rate: %.02f 1/day<br>" % params[1] +
        "Lin. growth rate: %.02f cm^3/day<br>" % params[2] +
        "<extra></extra>"),
    mode="lines"
))

# Define sliders for model parameters
initial_volume = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01,
    description='Initial tumour volume in cm^3:', continuous_update=True)
lambda_0 = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01,
    description='Exponential growth in 1/day:', continuous_update=True)
lambda_1 = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01,
    description='Linear growth in cm^3/day:', continuous_update=True)

# Define routine to update parameters
def update_parameters(change):
    parameters = [initial_volume.value, lambda_0.value, lambda_1.value]
    start_experiment = data['TIME in day'].min()
    end_experiment = data['TIME in day'].max()
    simulated_times = np.linspace(
        start=start_experiment, stop=end_experiment)
    simulated_data = model.simulate(parameters, simulated_times)
    with fig.batch_update():
        fig.data[0].y = simulated_data

# Connect update routine to sliders
initial_volume.observe(update_parameters, names="value")
lambda_0.observe(update_parameters, names="value")
lambda_1.observe(update_parameters, names="value")

# Group sliders
sliders = widgets.HBox([initial_volume, lambda_0, lambda_1])

widgets.VBox([sliders, fig])

## Bibliography

- <a name="ref1"> [1] </a> Eigenmann et. al., Combining Nonclinical Experiments with Translational PKPD Modeling to Differentiate Erlotinib and Gefitinib, Mol Cancer Ther (2016)
- <a name="ref1"> [2] </a> Koch et. al., Modeling of tumor growth and anticancer effects of combination therapy, Journal of Pharmacokinetics and Pharmacokinetics, (2009)
- <a name="ref1"> [3] </a> Hanahan & Weinberg, Hallmarks of Cancer, Cell, (2000)
- <a name="ref1"> [4] </a> Hanahan & Weinberg, Hallmarks of Cancer: The Next Generation, Cell, (2011)
- <a name="ref1"> [5] </a> Fouad & Aanei, Revisiting the hallmarks of cancer, American Journal of Cancer Research, (2017)

[Back to project overview](https://github.com/DavAug/ErlotinibGefitinib/blob/master/README.md) | [Back to lung cancer control growth overview](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/overview.ipynb) | [Forward to next notebook](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability_structural_model.ipynb)